In [ ]:
import matplotlib
matplotlib.use('QtAgg')

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [36]:
# =====================================
# Clase Agente
# =====================================
class AgenteOpinion:

    def __init__(self, id, opinion_inicial, peso=1.0):
        self.id       = id
        self.posicion = np.array(opinion_inicial, dtype=float)
        self.velocidad = np.zeros(3, dtype=float)
        self.peso     = peso
        self.historial = []

    def guardar_estado(self):
        self.historial.append(self.posicion.copy())

    def f_interaccion(self, d):
        atraccion = np.exp(-d**2 / SIGMA**2)
        repulsion = LAM * np.exp(-d**2 / RHO**2)
        return atraccion - repulsion

    def calcular_aceleracion(self, agentes):
        aceleracion = np.zeros(3)

        # Interacción de pares
        for agente in agentes:
            if agente.id != self.id:
                diff = agente.posicion - self.posicion
                d    = np.linalg.norm(diff)
                if d > 1e-10:
                    f = self.f_interaccion(d)
                    aceleracion += agente.peso * f / d * diff

        # Hipergrafo
        otros = [a for a in agentes if a.id != self.id]
        if len(otros) >= 2:
            contribucion_hiper = np.zeros(3)
            pares = 0
            for p in range(len(otros)):
                for q in range(p+1, len(otros)):
                    j, k  = otros[p], otros[q]
                    d_jk  = np.linalg.norm(k.posicion - j.posicion)
                    x_c   = (j.peso * j.posicion + k.peso * k.posicion) / (j.peso + k.peso)
                    phi   = 1.0 / (1.0 + d_jk)**ALPHA
                    contribucion_hiper += phi * (x_c - self.posicion)
                    pares += 1
            aceleracion += BETA * contribucion_hiper / pares

        return aceleracion

    def actualizar(self, agentes):
        a_actual   = self.calcular_aceleracion(agentes)
        self.posicion  += self.velocidad * dt + 0.5 * a_actual * dt**2
        a_nueva    = self.calcular_aceleracion(agentes)
        self.velocidad += 0.5 * (a_actual + a_nueva) * dt

In [37]:
# =====================================
# FUNCIÓN DE ENTROPÍA
# =====================================
def calcular_entropia_shannon(agentes, bins=10, rango=(-6, 6)):

    posiciones = np.array([a.posicion for a in agentes])

    indices = np.floor(
        (posiciones - rango[0]) / (rango[1] - rango[0]) * bins
    ).astype(int)

    indices = np.clip(indices, 0, bins - 1)

    celdas = indices[:, 0] * bins**2 + indices[:, 1] * bins + indices[:, 2]

    _, conteos = np.unique(celdas, return_counts=True)
    probabilidades = conteos / conteos.sum()

    H = -np.sum(probabilidades * np.log2(probabilidades + 1e-12))

    return H

In [39]:
import matplotlib
matplotlib.use('QtAgg')

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


# =====================================
# PARÁMETROS MANIPULABLES
# =====================================
N_AGENTES = 2
PASOS     = 1000
dt        = 0.05

SIGMA     = 2.5
RHO       = 3.0
LAM       = 0.5

ALPHA     = 2.0
BETA      = 0.1

OPINION_MIN = -1.0
OPINION_MAX =  1.0

PESO_MIN  = 0.5
PESO_MAX  = 1.5

SEMILLA   = None

# =====================================
# Inicialización
# =====================================
if SEMILLA is not None:
    np.random.seed(SEMILLA)

agentes = []

for i in range(N_AGENTES):
    opinion = np.random.uniform(OPINION_MIN, OPINION_MAX, 3)
    peso    = np.random.uniform(PESO_MIN, PESO_MAX)
    agentes.append(AgenteOpinion(i, opinion, peso))


# =====================================
# Simulación + Entropía
# =====================================
entropias = []

for t in range(PASOS):
    for agente in agentes:
        agente.guardar_estado()

    H = calcular_entropia_shannon(
        agentes,
        bins=10,
        rango=(OPINION_MIN*3, OPINION_MAX*3)
    )
    entropias.append(H)

    for agente in agentes:
        agente.actualizar(agentes)


# =====================================
# Colores
# =====================================
cmap   = plt.cm.get_cmap('tab10', N_AGENTES)
colors = [cmap(i) for i in range(N_AGENTES)]


# =====================================
# Figura animación 3D
# =====================================
fig_anim = plt.figure(figsize=(8, 6))
ax_anim  = fig_anim.add_subplot(111, projection='3d')
ax_anim.set_title("Dinámica de opiniones")
ax_anim.set_xlabel("Opinión X")
ax_anim.set_ylabel("Opinión Y")
ax_anim.set_zlabel("Opinión Z")

lineas, puntos = [], []

for i, agente in enumerate(agentes):
    color = colors[i]
    linea, = ax_anim.plot([], [], [], lw=2, color=color)
    punto, = ax_anim.plot([], [], [], 'o', color=color, markersize=8)
    lineas.append(linea)
    puntos.append(punto)

todos  = np.concatenate([np.array(a.historial) for a in agentes])
margin = 0.5

ax_anim.set_xlim(todos[:,0].min()-margin, todos[:,0].max()+margin)
ax_anim.set_ylim(todos[:,1].min()-margin, todos[:,1].max()+margin)
ax_anim.set_zlim(todos[:,2].min()-margin, todos[:,2].max()+margin)


def update(frame):
    for i, agente in enumerate(agentes):
        h = np.array(agente.historial)
        lineas[i].set_data(h[:frame+1, 0], h[:frame+1, 1])
        lineas[i].set_3d_properties(h[:frame+1, 2])
        puntos[i].set_data([h[frame, 0]], [h[frame, 1]])
        puntos[i].set_3d_properties([h[frame, 2]])
    return lineas + puntos


ani = FuncAnimation(fig_anim, update, frames=PASOS, interval=20, blit=False)

# =====================================
# Figura trayectorias completas
# =====================================
fig_full = plt.figure(figsize=(8, 6))
ax_full  = fig_full.add_subplot(111, projection='3d')
ax_full.set_title("Trayectorias completas de opinión")
ax_full.set_xlabel("Opinión X")
ax_full.set_ylabel("Opinión Y")
ax_full.set_zlabel("Opinión Z")

for i, agente in enumerate(agentes):
    h = np.array(agente.historial)
    color = colors[i]   # ← MISMO color que animación

    ax_full.plot(h[:, 0], h[:, 1], h[:, 2], lw=1.5, color=color)
    ax_full.scatter(h[0,0],  h[0,1],  h[0,2],  s=80, marker='o', color=color)
    ax_full.scatter(h[-1,0], h[-1,1], h[-1,2], s=80, marker='*', color=color)

ax_full.set_xlim(todos[:,0].min()-margin, todos[:,0].max()+margin)
ax_full.set_ylim(todos[:,1].min()-margin, todos[:,1].max()+margin)
ax_full.set_zlim(todos[:,2].min()-margin, todos[:,2].max()+margin)

# =====================================
# Gráfica de entropía
# =====================================
fig_ent, ax_ent = plt.subplots(figsize=(10, 4))
ax_ent.plot(entropias, linewidth=2)
ax_ent.set_xlabel("Paso temporal")
ax_ent.set_ylabel("Entropía de Shannon (bits)")
ax_ent.set_title("Entropía del sistema")
ax_ent.grid(True, alpha=0.3)
ax_ent.axhline(np.log2(N_AGENTES), linestyle='--',
               label=f'H máx = {np.log2(N_AGENTES):.2f} bits')
ax_ent.legend()

plt.tight_layout()
plt.show()

/var/folders/tf/29wfkhy14dd6f8m9h2hymqjh0000gn/T/ipykernel_2343/4091472179.py:68: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap   = plt.cm.get_cmap('tab10', N_AGENTES)
